# Titanic Dataset from Kaggle
- https://www.kaggle.com/datasets/yasserh/titanic-dataset/data

### About this file

The sinking of the Titanic is one of the most infamous shipwrecks in history.

On April 15, 1912, during her maiden voyage, the widely considered “unsinkable” RMS Titanic sank after colliding with an iceberg. Unfortunately, there weren’t enough lifeboats for everyone on board, resulting in the death of 1502 out of 2224 passengers and crew.

While there was some element of luck involved in surviving, it seems some groups of people were more likely to survive than others.

In this challenge, we ask you to build a predictive model that answers the question: “what sorts of people were more likely to survive?” using passenger data (ie name, age, gender, socio-economic class, etc).

#### Data:
 - PassengerId: Passenger Id 
 - Survived: O: No 1=Yes
 - Pclass: 1: 1st 2: 2nd 3 = 3rd
 - Name: Name of the Passenger
 - Sex: male or female
 - Age: Age in Years
 - SibSp: No. of siblings / spouses aboard the Titanic
 - Parch: No. of parents / children aboard the Titanic
 - Ticket: Ticket number
 - Fare: Passenger fare
 - Cabin: Cabin (Removed from final dataset)
 - Embarked: Port of Embarkation. C = Cherbourg, Q = Queenstown, S = Southampton U = Unknown


In [1]:
# run  make requirements first

# https://pypi.org/project/kagglehub/

import kagglehub

# Download latest version
path_titanic_dataset = kagglehub.dataset_download("yasserh/titanic-dataset")

print("Path to dataset files:", path_titanic_dataset)

/opt/anaconda3/envs/machine_learning/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /Users/leonardoomarbolanosrivera/.cache/kagglehub/datasets/yasserh/titanic-dataset/versions/1


In [2]:
import pandas as pd 

In [3]:
# Reading data and removing index col
df = pd.read_csv(f"{path_titanic_dataset}/Titanic-Dataset.csv")

In [4]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
# 891 rows and 12 cols
df.shape

(891, 12)

In [7]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [8]:

df.loc[df["Cabin"].isnull()].head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S


#### Cabing tiene demasiados NaN . Seleccionamos todas la columnas excepto Cabin

In [9]:
df=df[["PassengerId",	"Survived",	"Pclass",	"Name",	"Sex",	"Age",	"SibSp",	"Parch",	"Ticket",	"Fare",	"Embarked"]]

In [10]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


# En el caso de Age podemos usar la media para los datos nulos. 

In [11]:
df.loc[df["Age"].isnull()].head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,Q
17,18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,S
19,20,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,C
26,27,0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.2250,C
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.8792,Q


In [12]:
age_mean = df['Age'].mean()
age_mean
print("El valor de la Media de la columna es: %.2f" % age_mean)

El valor de la Media de la columna es: 29.70


In [13]:
#Asignación de Valores de Media a los Valores Nulos
df.loc[:, "Age"] =df.loc[:, 'Age'].fillna(age_mean)




In [14]:
df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       2
dtype: int64

# En el caso de Embarked  Podemos marcarlo con U para desconocido

In [15]:
df.loc[df["Embarked"].isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,NaN


In [16]:
df.loc[df["Embarked"].isnull(),"Embarked"]='U'


In [17]:
df.loc[df["Embarked"].isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked


In [18]:
df.Embarked.value_counts()

Embarked
S    644
C    168
Q     77
U      2
Name: count, dtype: int64

# Funcion sigmoide 

$$ \sigma(x) =  \frac{1}{1 + e^{-x}} $$

# Derivada de la funcion sigmoide
$$ \sigma'(x) =  \frac{e^{-x}}{(1 + e^{-x})^2} $$

# Otra forma de escribir la Derivada de la funcion sigmoide

$$ \sigma'(x) =  (\sigma(x))(1-\sigma(x)) $$

####  Tomado de https://interactivechaos.com/es/manual/tutorial-de-deep-learning/derivada-de-la-funcion-sigmoide

# La derivama mostrada arriba es lo mismo que 
$$ y' =  y(1 - y) $$

#### Tomado de https://es.wikipedia.org/wiki/Funci%C3%B3n_sigmoide




#### Markdown simbols cheatsheet.md  https://gist.github.com/LKS90/252ac41bd4a173be35b0

In [20]:
import math

In [26]:
def sigmoid(x):
    return (1 / (1 + math.exp(x)))

In [22]:
def derrivada_sigmoide_opt1(x):
    return (math.exp((-1 * x)) / (1 + math.exp((-1 *x ))**2))

In [23]:
def derrivada_sigmoide_opt2(x):
    """ Derivada de la función sigmoide """
    return sigmoid(x) * (1 - sigmoid(x))

In [24]:
derrivada_sigmoide_opt1(10)

4.539992966890862e-05

In [27]:
derrivada_sigmoide_opt2(10)

4.5395807735951673e-05

In [28]:
def derrivada_sigmoide(x):
    """ Derivada de la función sigmoide """
    var_sigmoid=sigmoid(x)
    return var_sigmoid * (1 - var_sigmoid)

In [29]:
derrivada_sigmoide(10)

4.5395807735951673e-05